In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/ibm-granite/granite-3.3-2b-instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ibm-granite/granite-3.3-2b-instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# ===========================
# 📌 STUDYMATE – SINGLE CELL PIPELINE (Google Colab)
# ===========================

!pip install transformers sentence-transformers faiss-cpu pymupdf

import fitz  # PyMuPDF
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from google.colab import files

# =====================================
# 1️⃣ Upload PDFs
# =====================================
print("📘 Upload your PDFs...")
uploaded = files.upload()

pdf_text = ""
for fname in uploaded.keys():
    doc = fitz.open(fname)
    for page in doc:
        pdf_text += page.get_text()

print("✅ PDF text extracted.")

# =====================================
# 2️⃣ Chunking Function
# =====================================
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(pdf_text)
print(f"🔹 Total chunks created: {len(chunks)}")

# =====================================
# 3️⃣ Load Embedding Model + Create FAISS Index
# =====================================
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("🔍 FAISS index ready.")

# =====================================
# 4️⃣ Load IBM Granite 3.3-2B Instruct
# =====================================

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "ibm-granite/granite-3.3-2b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

print("🤖 Granite Model loaded.")

# =====================================
# 5️⃣ Ask Questions (Conversational Q&A)
# =====================================

def ask_question(question, k=3):
    # embed question
    q_embed = embed_model.encode([question], convert_to_numpy=True)

    # search FAISS
    distances, indices = index.search(q_embed, k)

    # combine top chunks
    context = "\n".join([chunks[i] for i in indices[0]])

    # prepare prompt for Granite
    messages = [
        {"role": "system", "content": "You are StudyMate, an academic assistant."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=300)
    answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return answer

# =====================================
# 6️⃣ RUN THE Q&A LOOP
# =====================================

while True:
    q = input("\n❓ Ask a question (or type 'exit'): ")
    if q.lower() == "exit":
        break

    print("\n🧠 Answer:")
    print(ask_question(q))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 24.5 MB/s eta 0:00:00
